## Estudio de Convergencia: Calculos con PyGBe

### Tripsina 2tgd sometida a un Campo Electrico Externo

#### Parametros PyGBe

	Use GPU                 : 1
	P                       : 15
	threshold               : 0.50
	theta                   : 0.50
	NCRIT                   : 500
	CUDA block size         : 128
	Gauss points per element: 7
	Gauss points near singlr: 19
	1D Gauss points per side: 9
	GMRES tolerance         : 1e-08
	GMRES max iterations    : 1000
	GMRES restart iteration : 200

### Nanoshaper: 
#### Efield = 1e-12 y _k_ = 0.125

In [9]:
from numpy import *
from pylab import *
import os

file_data = open("convergencia_data_2tgd.txt","r")
data = loadtxt(file_data)
N = data[:5,0]  #N° Elementos
Esolv_ais_EF0 = data[:5,2]
Esurf_ais_EF0 = data[:5,3]
Esolv_int_EF0 = data[:5,4]
Esurf_int_EF0 = data[:5,5]
#Esurf_ais_EF0 = array([14, 14.5629, 14.5854, 14.6069, 14.6129])

print("Caso sistema aislado: ")
print("densidad: ", N)
print("Esolv: ",Esolv_ais_EF0)
r= N[3]/N[2]
print("r: ", r)
p_solv = log((Esolv_ais_EF0[3] - Esolv_ais_EF0[2])/(Esolv_ais_EF0[4] - Esolv_ais_EF0[3]))/log(r)
print("p_solv: ",p_solv)
fsolvanalitico = Esolv_ais_EF0[4] + ((Esolv_ais_EF0[4] - Esolv_ais_EF0[3])/((r**p_solv) - 1))
print('E_solv_extrapolado:',fsolvanalitico,'Kcal/mol')

print("Esurf: ",Esurf_ais_EF0)
p_surf = log((Esurf_ais_EF0[3] - Esurf_ais_EF0[2])/(Esurf_ais_EF0[4] - Esurf_ais_EF0[3]))/log(r)
print("p_surf: ",p_surf)
fsurfanalitico = Esurf_ais_EF0[4] + ((Esurf_ais_EF0[4] - Esurf_ais_EF0[3])/((r**p_surf) - 1))
print('E_surf_extrapolado:',fsurfanalitico,'Kcal/mol')

print("Caso sistema interactuando: ")
print("densidad: ", N)
print("Esolv: ",Esolv_int_EF0)
p_solv_int = log((Esolv_int_EF0[3] - Esolv_int_EF0[2])/(Esolv_int_EF0[4] - Esolv_int_EF0[3]))/log(r)
print("p_solv: ",p_solv_int)
fsolvanalitico_int = Esolv_int_EF0[4] + ((Esolv_ais_EF0[4] - Esolv_ais_EF0[3])/((r**p_solv_int) - 1))
print('E_solv_extrapolado:',fsolvanalitico_int,'Kcal/mol')

print("Esurf: ",Esurf_int_EF0)
p_surf_int = log((Esurf_int_EF0[3] - Esurf_int_EF0[2])/(Esurf_int_EF0[3] - Esurf_int_EF0[4]))/log(r)
print("p_surf: ",p_surf_int)
fsurfanalitico_int = Esurf_int_EF0[4] + ((Esurf_int_EF0[4] - Esurf_int_EF0[3])/((r**p_surf_int) - 1))
print('E_surf_extrapolado:',fsurfanalitico_int,'Kcal/mol')

Caso sistema aislado: 
densidad:  [  1.   2.   4.   8.  16.]
Esolv:  [-1201.802244  -859.653951  -758.635411  -724.900908  -711.750534]
r:  2.0
p_solv:  1.3591210765
E_solv_extrapolado: -703.349287761 Kcal/mol
Esurf:  [ 45.373319  45.37135   45.370463  45.369966  45.369687]
p_surf:  0.832980729746
E_surf_extrapolado: 45.3693299312 Kcal/mol
Caso sistema interactuando: 
densidad:  [  1.   2.   4.   8.  16.]
Esolv:  [-1200.5363    -859.172477  -758.187006  -724.44304   -711.309592]
p_solv:  1.36138382158
E_solv_extrapolado: -702.929901926 Kcal/mol
Esurf:  [ 46.040105  46.030576  46.005144  46.005976  46.005346]
p_surf:  0.40123169976
E_surf_extrapolado: 46.0033811485 Kcal/mol


In [10]:
#Calculo de Errores Relativos en Energia de Solvatacion y Superficial sistemas aislados:
PyGerr_solv = abs(Esolv_ais_EF0-fsolvanalitico)/abs(fsolvanalitico)
print(PyGerr_solv*100)
asympt_solv = N[-1]*PyGerr_solv[-1]/N

PyGerr_surf = abs(Esurf_ais_EF0-fsurfanalitico)/abs(fsurfanalitico)
print(PyGerr_surf*100)
asympt_surf = N[-1]*PyGerr_surf[-1]/N

figure(figsize=(3,2), dpi=80)
loglog(N[1:], PyGerr_solv[1:], c="k", marker="o", mfc="w", ms=3, ls="", lw=0.5, label="$E_{solv}$")
loglog(N[1:], asympt_solv[1:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
loglog(N[1:], PyGerr_surf[1:], c="k", marker="x", mfc="w", ms=3, ls="", lw=0.5, label="$E_{surf}$")
loglog(N[1:], asympt_surf[1:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
text(2.5e1,3e-6,"$E_{0}$ = 1e-12 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
ylabel('Error', fontsize=10)
xlabel('Densidad',fontsize=10)
subplots_adjust(left=0.25, bottom=0.25, right=0.96, top=0.96)
axis([1e-0,100,0.000001, 1e0])
legend(loc='best', fontsize = '6')
savefig('Error_2tgd_EsolvEsurf_ais_Ef1e-12_k0.125_nanoshaper.pdf')
clf()

[ 70.86848098  22.22290773   7.86040794   3.0641419    1.19446289]
[ 0.00879243  0.0044525   0.00249743  0.00140198  0.00078703]


In [11]:
#Calculo de Errores Relativos en Energia de Solvatacion y Superficial casos interactuando:
PyGerr_solv = abs(Esolv_int_EF0-fsolvanalitico_int)/abs(fsolvanalitico_int)
print(PyGerr_solv*100)
asympt_solv = N[-1]*PyGerr_solv[-1]/N

PyGerr_surf = abs(Esurf_int_EF0-fsurfanalitico_int)/abs(fsurfanalitico_int)
print(PyGerr_surf*100)
asympt_surf = N[-1]*PyGerr_surf[-1]/N

figure(figsize=(3,2), dpi=80)
loglog(N[1:], PyGerr_solv[1:], c="k", marker="o", mfc="w", ms=3, ls="", lw=0.5, label="$E_{solv}$")
loglog(N[1:], asympt_solv[1:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
loglog(N[1:], PyGerr_surf[1:], c="k", marker="x", mfc="w", ms=3, ls="", lw=0.5, label="$E_{surf}$")
loglog(N[1:], asympt_surf[1:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
text(2.5e1,3e-6,"$E_{0}$ = 1e-12 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
ylabel('Error', fontsize=10)
xlabel('Densidad',fontsize=10)
subplots_adjust(left=0.25, bottom=0.25, right=0.96, top=0.96)
axis([1e-0,100,0.000001, 1e0])
legend(loc='best', fontsize = '6')
savefig('Error_2tgd_EsolvEsurf_int_Ef1e-12_k0.125_nanoshaper.pdf')
clf()

[ 70.79033012  22.22733371   7.86096934   3.06049551   1.19210892]
[ 0.07982859  0.0591149   0.003832    0.00564057  0.0042711 ]


In [23]:
################################################################################################################################

#### Efield = 2.3763e-4 y _k_ = 0.125

In [24]:
##tol = 1e-5; K = 4; P = 6
N = data[5:,0]  #N° Elementos
Esolv_ais_EF1 = data[5:,2]
Esurf_ais_EF1 = data[5:,3]
Esolv_int_EF1 = data[5:,4]
Esurf_int_EF1 = data[5:,5]
#Esurf_ais_EF0 = array([14, 14.5629, 14.5854, 14.6069, 14.6129])

print("Caso sistema aislado: ")
print("densidad: ", N)
print("Esolv: ",Esolv_ais_EF1)
r= N[3]/N[2]
print("r: ", r)
p_solv = log((Esolv_ais_EF1[2] - Esolv_ais_EF1[1])/(Esolv_ais_EF1[3] - Esolv_ais_EF1[2]))/log(r)
print("p_solv: ",p_solv)
fsolvanalitico = Esolv_ais_EF1[3] + ((Esolv_ais_EF1[3] - Esolv_ais_EF1[2])/((r**p_solv) - 1))
print('E_solv_extrapolado:',fsolvanalitico,'Kcal/mol')

print("Esurf: ",Esurf_ais_EF1)
p_surf = log((Esurf_ais_EF1[2] - Esurf_ais_EF1[1])/(Esurf_ais_EF1[3] - Esurf_ais_EF1[2]))/log(r)
print("p_surf: ",p_surf)
fsurfanalitico = Esurf_ais_EF1[3] + ((Esurf_ais_EF1[3] - Esurf_ais_EF1[2])/((r**p_surf) - 1))
print('E_surf_extrapolado:',fsurfanalitico,'Kcal/mol')

print("\n")
print("Caso sistema interactuando: ")
print("densidad: ", N)
print("Esolv: ",Esolv_int_EF1)
p_solv_int = log((Esolv_int_EF1[2] - Esolv_int_EF1[1])/(Esolv_int_EF1[3] - Esolv_int_EF1[2]))/log(r)
print("p_solv: ",p_solv_int)
fsolvanalitico_int = Esolv_int_EF1[3] + ((Esolv_ais_EF1[3] - Esolv_ais_EF1[2])/((r**p_solv_int) - 1))
print('E_solv_extrapolado:',fsolvanalitico_int,'Kcal/mol')

print("Esurf: ",Esurf_int_EF1)
p_surf_int = log((Esurf_int_EF1[2] - Esurf_int_EF1[1])/(Esurf_int_EF1[3] - Esurf_int_EF1[2]))/log(r)
print("p_surf: ",p_surf_int)
fsurfanalitico_int = Esurf_int_EF1[3] + ((Esurf_int_EF1[3] - Esurf_int_EF1[2])/((r**p_surf_int) - 1))
print('E_surf_extrapolado:',fsurfanalitico_int,'Kcal/mol')

Caso sistema aislado: 
densidad:  [ 1.  2.  4.  8.]
Esolv:  [-1268.8663    -974.664607  -873.944131  -840.328589]
r:  2.0
p_solv:  1.58315669014
E_solv_extrapolado: -823.489221213 Kcal/mol
Esurf:  [-145.019981 -145.047268 -145.047575 -145.068203]
p_surf:  -6.07022148487
E_surf_extrapolado: -145.047263362 Kcal/mol


Caso sistema interactuando: 
densidad:  [ 1.  2.  4.  8.]
Esolv:  [-1263.383121  -973.246687  -872.612051  -838.840286]
p_solv:  1.57523744583
E_solv_extrapolado: -821.861413155 Kcal/mol
Esurf:  [-147.950945 -148.003994 -148.018911 -148.048856]
p_surf:  -1.00535771203
E_surf_extrapolado: -147.98918718 Kcal/mol


In [25]:
#Calculo de Errores Relativos en Energia de Solvatacion y Superficial sistemas aislados:
PyGerr_solv = abs(Esolv_ais_EF1-fsolvanalitico)/abs(fsolvanalitico)
print(PyGerr_solv*100)
asympt_solv = N[-1]*PyGerr_solv[-1]/N

PyGerr_surf = abs(Esurf_ais_EF1-fsurfanalitico)/abs(fsurfanalitico)
print(PyGerr_surf*100)
asympt_surf = N[-1]*PyGerr_surf[-1]/N

figure(figsize=(3,2), dpi=80)
loglog(N[:], PyGerr_solv[:], c="k", marker="o", mfc="w", ms=3, ls="", lw=0.5, label="$E_{solv}$")
loglog(N[:], asympt_solv[:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
loglog(N[:], PyGerr_surf[:], c="k", marker="x", mfc="w", ms=3, ls="", lw=0.5, label="$E_{surf}$")
loglog(N[:], asympt_surf[:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
text(2.5e1,3e-6,"$E_{0}$ = 2e-4 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
ylabel('Error', fontsize=10)
xlabel('Densidad',fontsize=10)
subplots_adjust(left=0.25, bottom=0.25, right=0.96, top=0.96)
axis([1e-0,100,0.000001, 1e0])
legend(loc='best', fontsize = '6')
savefig('Error_2tgd_EsolvEsurf_ais_Ef2e-4_k0.125_nanoshaper.pdf')
clf()

[ 54.08414188  18.35790705   6.12696663   2.04488017]
[  1.88092911e-02   3.19758527e-06   2.14852733e-04   1.44364240e-02]


In [26]:
#Calculo de Errores Relativos en Energia de Solvatacion y Superficial casos interactuando:
PyGerr_solv = abs(Esolv_int_EF1-fsolvanalitico_int)/abs(fsolvanalitico_int)
print(PyGerr_solv*100)
asympt_solv = N[-1]*PyGerr_solv[-1]/N

PyGerr_surf = abs(Esurf_int_EF1-fsurfanalitico_int)/abs(fsurfanalitico_int)
print(PyGerr_surf*100)
asympt_surf = N[-1]*PyGerr_surf[-1]/N

figure(figsize=(3,2), dpi=80)
loglog(N[:], PyGerr_solv[:], c="k", marker="o", mfc="w", ms=3, ls="", lw=0.5, label="$E_{solv}$")
loglog(N[:], asympt_solv[:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
loglog(N[:], PyGerr_surf[:], c="k", marker="x", mfc="w", ms=3, ls="", lw=0.5, label="$E_{surf}$")
loglog(N[:], asympt_surf[:], c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
text(2.5e1,3e-6,"$E_{0}$ = 2e-4 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
ylabel('Error', fontsize=10)
xlabel('Densidad',fontsize=10)
subplots_adjust(left=0.25, bottom=0.25, right=0.96, top=0.96)
axis([1e-0,100,0.000001, 1e0])
legend(loc='best', fontsize = '6')
savefig('Error_2tgd_EsolvEsurf_int_Ef2e-4_k0.125_nanoshaper.pdf')
clf()

[ 53.72216055  18.41980551   6.17508464   2.06590461]
[ 0.0258412   0.01000534  0.02008513  0.04031972]
